In [1]:
from utils.dataloader import dtLoader
import numpy as np
import pywt
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize

In [2]:
source = "/Users/pradithaalwis/Projects/University of Peradeniya/Fetal Mov Data/Ward/"
drop_zero_path = "/Users/pradithaalwis/Projects/University of Peradeniya/Fetal Mov Data//Randomly_Dropped_Windows/"
sensors = ['ax1', 'ay1', 'az1', 'ax2', 'ay2', 'az2', 'ax3', 'ay3', 'az3', 'ax4', 'ay4', 'az4']
sample_freq = 32

# Stride in seconds
stride = 1

# Window length in seconds
window_length = 8

# Safety margin in seconds
front_margin = 2
rear_margin = 2

In [3]:
"""
Splits the signals into windows
Args:
    SOURCE          : path to the sensor data
    SAMPLE_FREQ     : sample frequency of sensors
    STRIDE          : stride used for windowing (in seconds)
    WINDOW_LENGTH   : length of the required window in seconds
    FRONT_MARGIN    : Required offset before a fetal kick (in seconds)
    REAR_MARGIN     : Required offset after a fetal kick (in seconds)
"""
dataloader = dtLoader(SOURCE=source,
                        SAMPLE_FREQ=sample_freq,
                        STRIDE=stride,
                        WINDOW_LENGTH=window_length,
                        FRONT_MARGIN=front_margin,
                        REAR_MARGIN=rear_margin,
                        SENSORS=sensors)

In [4]:
"""
Args:
Returns:
    self.window_list            : Set of generated windows
    self.kick_count_list        : Number of kicks in each window
    counts                      : Dictionary containing number of windows                             with the given number of kicks
"""
counts = dataloader.split_windows()

--------------------------------------------------------------------------------
These are the list of files in the source directory.
['W17_R1.csv', 'W09_R1.csv', 'W05_R1.csv', 'W21_R1.csv', 'W05_R2.csv', 'W21_R2.csv', 'W15_R2.csv', 'W19_R1.csv', 'W07_R1.csv', 'W15_R1.csv', 'W23_R1.csv', 'W11_R3.csv', 'W27_R2.csv', 'W11_R2.csv', 'W27_R1.csv', 'W11_R1.csv', 'W03_R1.csv', 'W25_R1.csv', 'W11_R5.csv', 'W29_R1.csv', 'W01_R1.csv', 'W11_R4.csv', 'W13_R1.csv', 'W25_R2.csv', 'W29_R2.csv', 'W29_R3.csv', 'W25_R3.csv', 'W04_R2.csv', 'W20_R1.csv', 'W08_R1.csv', 'W16_R1.csv', 'W04_R1.csv', 'W22_R1.csv', 'W30_R1.csv', 'W06_R1.csv', 'W18_R1.csv', 'W14_R1.csv', 'W22_R2.csv', 'W10_R1.csv', 'W02_R1.csv', 'W26_R1.csv', 'W24_R3.csv', 'W24_R2.csv', 'W12_R1.csv', 'W24_R1.csv', 'W28_R1.csv']
--------------------------------------------------------------------------------
Length of dataframe :  1286  seconds
W17_R1.csv has been processed
Length of dataframe :  1263  seconds
W09_R1.csv has been processed
Length

In [5]:
window_list, num_kick_list = dataloader.drop_zero_kicks(counts[0] - counts[1], drop_zero_path)

In [6]:
print(window_list.shape)

(3529, 12, 256)


In [7]:
def gen_scalo(signal):
    values = np.geomspace(1, 32, num=64)
    widths = []
    sample_freq = 32
    frequencies = pywt.scale2frequency('cmor2.5-0.5', values) * sample_freq

    for i in range(len(values)):
        if frequencies[i] > 2:
            widths.append(values[i])
    C = 0.5
    B = 2.5
    cwtmatr, freqs = pywt.cwt(window_list[0][2], widths, 'cmor' + str(B) + '+' + str(C), sampling_period= 1 / sample_freq, method='fft')
    cwtmatr = np.abs(cwtmatr[:-1, :-1]) ** 2
    return cwtmatr, freqs

In [8]:
# Should randomize this selection
scalograms = []
for i in range(len(num_kick_list)):
    temp = []
    for sensor in window_list[i]:
        cwtmatr, freq = gen_scalo(sensor)
        normed_matrix = normalize(cwtmatr, axis=1, norm='l1')
        temp.append(cwtmatr)
    temp = np.array(temp, dtype=np.float32)
    scalograms.append(temp)
scalograms = np.array(scalograms, dtype=np.float32)

In [9]:
print(min(num_kick_list))

0


In [10]:
from torch.utils.data import TensorDataset, DataLoader, Dataset
from sklearn.model_selection import train_test_split
import torch

In [11]:
scalograms_0 = []
scalograms_1 = []
scalograms_2 = []
scalograms_3 = []
for i in range(len(scalograms)):
    match num_kick_list[i]:
        case 0:
            scalograms_0.append(scalograms[i])
        case 1:
            scalograms_1.append(scalograms[i])
        case 2:
            scalograms_2.append(scalograms[i])
        case 3:
            scalograms_3.append(scalograms[i])
        case _:
            print("Unknown Class!!!")
del scalograms

In [12]:
X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(scalograms_0, [0 for i in range(len(scalograms_0))], test_size=0.2)
del scalograms_0
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(scalograms_1, [1 for i in range(len(scalograms_1))], test_size=0.2)
del scalograms_1
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(scalograms_2, [2 for i in range(len(scalograms_2))], test_size=0.2)
del scalograms_2
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(scalograms_3, [3 for i in range(len(scalograms_3))], test_size=0.2)
del scalograms_3

In [13]:
print(len(X_train_0), len(X_train_1), len(X_train_2), len(X_train_3))

1384 1252 165 21


In [14]:
X_train = X_train_0
del X_train_0
X_train.extend(X_train_1)
del X_train_1
X_train.extend(X_train_2)
del X_train_2
X_train.extend(X_train_3)
del X_train_3

y_train = y_train_0
del y_train_0
y_train.extend(y_train_1)
del y_train_1
y_train.extend(y_train_2)
del y_train_2
y_train.extend(y_train_3)
del y_train_3

X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)

In [15]:
X_test = X_test_0
del X_test_0
X_test.extend(X_test_1)
del X_test_1
X_test.extend(X_test_2)
del X_test_2
X_test.extend(X_test_3)
del X_test_3

y_test = y_test_0
del y_test_0
y_test.extend(y_test_1)
del y_test_1
y_test.extend(y_test_2)
del y_test_2
y_test.extend(y_test_3)
del y_test_3

X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)

In [16]:
print(X_test.shape, y_test.shape)

(707, 12, 37, 255) (707,)


In [17]:
# Convert data to torch tensors
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.longlong))
        self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len

In [18]:
import torch
import torchvision
from torchvision import transforms
class CustomToTensor(object):
    """Convert ndarrays in sample to Tensors."""
    def __call__(self, sample):
        # Swap color axis because
        # numpy image: H x W x C
        # torch image: C x H x W
        print(sample[1].shape, sample[1])
        return (torch.from_numpy(sample[0]).to(torch.float32).to("mps"), torch.from_numpy(sample[1]).to(torch.long))
class CustomNormalize(object):
    """Normalize an image with mean and standard deviation."""
    def __init__(self, mean, std):
        self.mean = torch.tensor(mean).to(torch.float32).to("mps")
        self.std = torch.tensor(std).to(torch.float32).to("mps")

    def __call__(self, tensor):
        """
        Args:
            tensor (Tensor): Tensor image of size (C, H, W) to be normalized.

        Returns:
            Tensor: Normalized image.
        """
        for t, m, s in zip(tensor, self.mean, self.std):
            t.sub_(m).div_(s)
        return tensor
transform = transforms.Compose(
    [CustomToTensor(),
    CustomNormalize(mean=[0.5] * 12, std=[0.5] * 12)])

In [19]:
train_data = Data(X_train, y_train)
test_data = Data(X_test, y_test)

In [24]:
batch_size = 10
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)#, num_workers=2)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False)#, num_workers=2)

In [67]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        num_channels = 12
        self.conv1 = nn.Conv2d(num_channels, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 6 * 60, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [68]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [69]:
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training


In [71]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
# imshow(torchvision.utils.make_grid(images))
print(labels)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [72]:
outputs = net(images)

In [73]:
_, predicted = torch.max(outputs, 1)

In [74]:
print(predicted)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [75]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [76]:
print(correct, total)

313 707


In [77]:
print((correct / total) * 100)

44.271570014144274
